# Findings on Random Forest Reweighting
Here are some findings from my experiments with using Random Forests to perform reweighting

## Principle
We wish to "reweight" a sample of events, called the "source", in order to correctly represent another sample of events, called the "target".

In some region, $i$ of feature-space, consider the set of weights, $w_i$:
$$ w_i = \frac{n_{t,i}}{n_{s,i}}$$
where $n_{t,i}$ is the number of events from the target sample in region $i$, and $n_{s,i}$ the corresponding number for the source sample. 

If the source and target samples were _i.i.d_ with equal numbers of events in each, then $w_i = 1$ as $n_{s,i}, n_{t,i}\to\infty$. In the case that the two samples were drawn from different underlying p.d.f.s, then $w_i \neq 1$. In this case, statistical quantities calculated within region $i$ from the source sample could be corrected to represent those quantities as calculated from the target sample within region $i$, if each event of the source sample were weighted by $w_i$. Applying these weights is commonly referred to as "reweighting", since the events may already have weights (e.g. from the Monte-Carlo simulation process). As a trivial example, consider the number of events within a region $i$:
$$ \hat{n_{t,i}} = w_i n_{s,i} = \frac{n_{t,i}}{n_{s,i}}.n_{s,i} = n_{t,i}$$

To correct the statistical properties of the entire sample, feature-space must be partitioned with the goal of identifying regions where $w_i$ is farthest from $n_t/n_s$. Events falling in these regions can then be weighted to remove the discrepancies they represent.

Decision trees offer a natural way to do this, since they are designed to partition feature-space into disjoint subspaces. In the case of classification, these are generally the subspaces that are as pure in a single class (class homogeneous) as possible. For reweighting, the regions  identified are as pure as possible in either source or target events.

Many implementations of decision trees do not return the $w_i$, but rather the probability of being in one class or another. The two are related as follows:
$$
\begin{align}
p_{s, i} &= \frac{n_{s,i}}{n_{t,i}+n_{s,i}}\\
\frac{n_{t,i}}{n_{s,i}} + 1 &= \frac{1}{p_{s,i}}\\
w_{i} &= \frac{1}{p_{s,i}} - 1\\
\end{align}
$$
where $p_{s}$ denotes the probability the event comes from the source distribution. This equation intuitively makes sense: if there are few target events in region $i$ relative to the source sample, then the source events should get a small weight. In this case, $p_{i}$ would be large, so $1/p_{i}$ would be large and hence $w_i$ small, as required. Conversely if there are more target events than source events, the $w_i$ would be large as expected. 

There are two manifestations of completely pure nodes. If $n_{t,i} = 0$ and $n_{s,i} \neq 0$, then $p_{s,i} = 1$. This yields a weight of zero, which is unproblematic. On the other hand, if $n_{s,i} = 0$ and $n_{t,i} \neq 0$, then $p_{s,i} = 0$ This will yield an infinite $1/p_{s, i}$ and hence $w_i$. Problems with `NaN` can be avoided by adding an infinitesimal constant, $\epsilon$, to the formula:
$$
w_{i} = \frac{1}{p_{s,i}+\epsilon} - 1
$$, yielding an extremely large, but finite $w_{i}$.

Assuming there is support between the two datasets, it is extremely unlikely that $p_{s,i} = 1$ when calculated over the entire ensemble since this would require an observation to be in a pure source node in every member of the ensemble. However, in the individual trees, this is possible if the tree is highly developed given the fluctuations in dataset due to bootstrapping (~37% of observations will not have previously been observed by this tree). In this case, any statistical property calculated using a single tree's weights will be ruined by the presence of events with spuriously high weights.

Four possible methods to deal with this:
1. Do not allow the tree to form pure nodes.
2. Keep $\epsilon$ fairly large, to avoid large $w_{t, i}$ and hence large impact on stastical calculations
3. Treat cases of $p_{t,i}$ specially: simply assign $w_{t, i} = 0$ or 1?
4. Never use individual $w_{t, i}$.

Solution 1 may lead to problems of bias in central prediction, if tree cannot develop sufficiently flexibly to achieve a good model. Perhaps this could be addressed by increasing the number of members in the ensemble.
Solution 2 is not a real solution, since if $\epsilon$ is too large it will bias results where $p_{t,i} \neq 1$ and $p_{t,i}\neq 0$. Solution 3 will also lead to biases. Solution 4 would work, but does not permit the use of individual trees for estimation of the variance in the weights.

The answer to the problem raised in solution 4 would be to bootstrap the random forest (or *any* other estimator) and use this to estimate uncertainties.